<a href="https://colab.research.google.com/github/gnoparus/bualabs/blob/master/nbs/16b-neural-networks-initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural Network 102 - Initialization Neural Network

เปรียบเทียบการ Initialize Weight แบบต่าง ๆ ว่าจะมีผลต่อ Gradient อย่างไร แบบไหนช่วยลดปัญหา Vanishing Gradient และ Exploding Gradient

# 0. Magic

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [0]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch
from torch import tensor

# 2. Download Dataset

เคสนี้เราจะใช้ MNIST Dataset เหมือนเดิม

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

ดาวน์โหลดชุดข้อมูลมาใส่ x, y ทั้ง Training Set และ Validation Set

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

# 3. เตรียมข้อมูล

หา mean และ standard deviation ของข้อมูล Training Set

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

เนื่องจากทุกอย่างมาจากข้อมูลคนละแหล่ง เราไม่ทราบว่าอันไหนสำคัญกว่าอันไหน เราจึง Normalize ข้อมูลทุกอย่างให้อยู่ใน Scale เดียวกันให้หมด โดย mean = 0, std = 1 เท่ากันหมด แล้วให้โมเดลเรียนรู้เอง จากข้อมูลตัวอย่างว่าอันไหน สำคัญเท่าไร สัมพันธ์กันอย่างไร

Normalize คือการทำให้ข้อมูล มี mean = 0 และ standard deviation (std) = 1 ด้วยการ ลบด้วย mean หารด้วย std

In [0]:
# x = data, m = mean, s = standard deviation
def normalize(x, m, s): 
    return (x-m)/s

In [0]:
x_train = normalize(x_train, train_mean, train_std)

Normalize ข้อมูล Validation Set ด้วย mean, std ของ Training Set เพื่อปรับให้เป็น Scale เดียวกัน

In [0]:
x_valid = normalize(x_valid, train_mean, train_std)

ลองดูค่า mean, std หลังจาก Normalize เรียบร้อยแล้ว

In [10]:
train_mean_after, train_std_after = x_train.mean(), x_train.std()
train_mean_after, train_std_after

(tensor(0.0001), tensor(1.))

ได้ mean เป็นเลขจุดทศนิยมน้อยมาก ๆ ใกล้เคียง 0 และ std ใกล้เคียง 1

n = จำนวน Record, m จำนวน Feature, c = จำนวน class

In [11]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

แปลงจาก Long เป็น Float ก่อนส่งให้ Loss Function เนื่องจากเราใช้ Mse แทน Cross Entropy

In [0]:
y_train, y_valid = y_train.float(), y_valid.float()

# 4. Model

เราจะสร้าง 4 Hidden Layers Neural Networks ที่มี 100 Neuron ในแต่ละ Hidden Layer 

In [0]:
# nh = number of hidden
nh = 100

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    def forward(self, inp):
        raise Exception('Not implemented')
    def backward(self):
        self.bwd(self.out, *self.args)        

In [0]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.)# - 0.5
    def bwd(self, out, inp):
        inp.g = (inp>0).float()*out.g

In [0]:
class Sigmoid(Module):
    def forward(self, inp):
        return 1/(1 + torch.exp(-inp)) 
    def bwd(self, out, inp):
        inp.g = (self.forward(inp) * (1 - self.forward(inp))) * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, inp):
        return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.squeeze(-1).t() @ out.g
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze(-1) - targ).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2. * (inp.squeeze(-1) - targ).unsqueeze(-1) / inp.shape[0]        

สร้างโมเดล ที่มี 4 Hidden Layer และ Activation หลายแบบให้เลือก ใน Comment

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2), Relu(), Lin(w3, b3), Relu(), Lin(w4, b4)]
#         self.layers = [Lin(w1, b1), Sigmoid(), Lin(w2, b2), Sigmoid(), Lin(w3, b3), Sigmoid(), Lin(w4, b4)]
#         self.layers = [Lin(w1, b1), Lin(w2, b2), Lin(w3, b3), Lin(w4, b4)]
        self.loss = Mse()
    def __call__(self, x, targ):
#         set_trace()
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ), x
    def backward(self):
        self.loss.backward()        
        for l in reversed(self.layers):
            l.backward()

# 5. Weight and Bias Initialization

เราจะลอง Initinalize กำหนดค่าเริ่มต้น Weight และ Bias แบบต่าง ๆ แล้วเราจะใช้ข้อมูลเดียวกัน รันผ่านโมเดล แล้ว Backpropagate เพื่อเปรียบเทียบ ว่า Gradient เป็นอย่างไร  

## 5.1 Normal Distribution (Gaussian)

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วยค่า Random แบบ Normal Distribution (Gaussian)

In [0]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh)
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh)
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1)
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [21]:
w1.mean(), w1.std()

(tensor(-0.0028), tensor(1.0025))

In [22]:
w2.mean(), w2.std()

(tensor(0.0061), tensor(0.9916))

In [23]:
w3.mean(), w3.std()

(tensor(-0.0044), tensor(1.0063))

In [24]:
w4.mean(), w4.std()

(tensor(-0.1578), tensor(0.9961))

Weight ดูดีมาก ใกล้เคียง mean = 0, std = 1 แต่ไม่มีประโยชน์ เพราะเราสนใจแต่ Gradient

### ลองสมมติว่าเป็น Layer แรก

In [25]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(-1.1228), tensor(27.2951))

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [26]:
z1 = (x_train @ w1).clamp(0.)-0.5
z1.mean(), z1.std()

(tensor(9.7059), tensor(15.7963))

เราได้ Activation ของ Layer แรก ที่ไม่ใกล้เคียง mean = 0, std = 1 เลย อาจจะมีปัญหาตอนเทรนได้

### สร้างโมเดล รัน forward และ backward 

In [0]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [28]:
w4.g.mean(), w4.g.std()

(tensor(-3221742.), tensor(2968939.5000))

In [29]:
w3.g.mean(), w3.g.std()

(tensor(46754.3906), tensor(401328.8125))

In [30]:
w2.g.mean(), w2.g.std()

(tensor(17742.5352), tensor(295852.2812))

In [31]:
w1.g.mean(), w1.g.std()

(tensor(124.9641), tensor(69403.4297))

Gradient ที่เราต้องการคือประมาณ mean = 0, std = 1 แต่นี่ไม่ได้ใกล้เคียงเลย ถ้าเป็นแบบนี้จะทำให้เกิดปัญหา Exploding Gradient

## 5.2 Xavier Initialization (หรืออีกชื่อคือ Glorot initialization.)

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วย Xavier Initialization โดยสำหรับ ReLU แล้ว gain = sqrt(2)

In [0]:
gain = math.sqrt(2.)

w1 = torch.randn(m, nh) * math.sqrt(2./(m+nh)) * gain
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh) * math.sqrt(2./(nh+nh)) * gain
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh) * math.sqrt(2./(nh+nh)) * gain
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1) * math.sqrt(2./(nh+1)) * gain
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [33]:
w1.mean(), w1.std()

(tensor(0.0002), tensor(0.0672))

In [34]:
w2.mean(), w2.std()

(tensor(0.0006), tensor(0.1414))

In [35]:
w3.mean(), w3.std()

(tensor(-0.0005), tensor(0.1423))

In [36]:
w4.mean(), w4.std()

(tensor(-0.0281), tensor(0.2342))

Weight จะ mean, std เป็นอย่างไรเรายังไม่สนใจ

### ลองสมมติว่าเป็น Layer แรก

In [37]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(0.0563), tensor(1.7842))

เราได้ Output ของ Layer แรก ก่อน Activation ที่ใกล้เคียง mean = 0, std = 1 มากขึ้น

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [38]:
z1 = (x_train @ w1).clamp(0.)#-0.5
z1.mean(), z1.std()

(tensor(0.7332), tensor(1.0649))

ใกล้เคียง mean = 0, std = 1

### สร้างโมเดล

In [0]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [40]:
w4.g.mean(), w4.g.std()

(tensor(-11.8426), tensor(13.2116))

In [41]:
w3.g.mean(), w3.g.std()

(tensor(0.2225), tensor(1.9578))

In [42]:
w2.g.mean(), w2.g.std()

(tensor(0.1789), tensor(1.3132))

In [43]:
w1.g.mean(), w1.g.std()

(tensor(0.0073), tensor(0.7052))

เราได้ Gradient ที่ใกล้เคียง mean = 0, std = 1 มากขึ้น

## 5.3 Kaiming Initialization 

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วย Kaiming Initialization ที่จะ Normalize ให้ Weight มี Varience ขึ้นกับจำนวน Input ของ Neuron 

In [0]:
gain = math.sqrt(2.) # ReLU

w1 = torch.randn(m, nh) * math.sqrt(2./m) * gain
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh) * math.sqrt(2./nh) * gain
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh) * math.sqrt(2./nh) * gain
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1) * math.sqrt(2./nh) * gain
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [45]:
w1.mean(), w1.std()

(tensor(-6.6740e-05), tensor(0.0720))

In [46]:
w2.mean(), w2.std()

(tensor(-0.0001), tensor(0.1983))

In [47]:
w3.mean(), w3.std()

(tensor(0.0006), tensor(0.1997))

In [48]:

w4.mean(), w4.std()

(tensor(-0.0105), tensor(0.1900))

Weight จะ mean, std เป็นอย่างไรเรายังไม่สนใจ

### ลองสมมติว่าเป็น Layer แรก

In [49]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(-0.0764), tensor(1.9336))

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [50]:
z1 = (x_train @ w1).clamp(0.)-0.5
z1.mean(), z1.std()

(tensor(0.2223), tensor(1.0811))

เราได้ Activation ของ Layer แรก ที่ใกล้เคียง mean = 0, std = 1 มากขึ้นอีก

### สร้างโมเดล

In [0]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [52]:
w4.g.mean(), w4.g.std()

(tensor(-11.1901), tensor(12.1440))

In [53]:
w3.g.mean(), w3.g.std()

(tensor(0.0443), tensor(1.0990))

In [54]:
w2.g.mean(), w2.g.std()

(tensor(0.0118), tensor(0.8286))

In [55]:
w1.g.mean(), w1.g.std()

(tensor(0.0027), tensor(0.6108))

เราได้ Gradient ที่ใกล้เคียง mean = 0, std = 1 มากขึ้นอีก

# 6. สรุป

1. วิธีการ Initialize ค่าของ Weight มีความสำคัญอย่างมาก ต่อ Gradient ซึ่งมีผลต่อการเทรน Neural Network
1. ยิ่ง Deep Neural Network ยิ่ง Deep มากขึ้นเท่าไร ยิ่งส่งผลต่อ Gradient เป็นทวีคูณ ถ้าน้อยก็จะน้อยไปเรื่อย ๆ จนเป็น 0 ถ้ามากก็จะมากจนเกินรับไหว เป็น Not a number (nan)
1. ในการเทรน Neural Network แล้ว Gradient เป็นส่วนสำคัญที่สุด ที่ช่วยให้เทรนสำเร็จ ได้โมเดลที่ดี จน Yann LeCun เสนอให้เรียกว่า Differentiable Programming แทน Deep Learning (Gradient คือ Diff ของหลายตัวแปร เช่น Matrix, Tensor)
1. Xavier / Glorot Initialization และ Kaiming / He Initialization มี Variation อื่น ๆ อีก สำหรับโมเดล Convolution Neural Network (CNN) หรือ Fully Connected Layer, การ Random แบบ Uniform Distribution หรือ Normal Distribution และ Activation ต่าง ๆ เช่น Sigmoid, ReLU, Tanh จะมีอัลกอริทึมแตกต่างกันนิดหน่อย จะอธิบายต่อไป

# Credit

* https://course.fast.ai/videos/?lesson=8
* https://pytorch.org/docs/stable/nn.init.html
* https://arxiv.org/abs/1502.01852
* http://proceedings.mlr.press/v9/glorot10a.html
* https://deeplearning.net